**annotationDeprecationOmission.ipynb**

Omission of properties required for deprecated concepts.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/annotationDeprecationOmission.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [1]:
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...done!


Define SPARQL query to retrieve ID, term, and properties that are reservd for use on deprecated concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various elements below).

**NB.2:** A concept is considered deprecated if it has a property ``<oboInOwl:inSubset>`` whose value contains the string ``obsolete``, e.g. ``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#obsolete"/>`` or``<oboInOwl:inSubset>obsolete/>``.


In [2]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?obsolete_since ?subsetpath ?deprecated ?replacedBy ?consider ?oldParent ?deprecation_comment WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id :obsolete_since ?obsolete_since .}
OPTIONAL {?id owl:deprecated ?deprecated .}
OPTIONAL {?id oboInOwl:replacedBy ?replacedBy .}
OPTIONAL {?id oboInOwl:consider ?consider .}
OPTIONAL {?id :oldParent ?oldParent .}
OPTIONAL {?id :deprecation_comment ?deprecation_comment .}
}
"""

# Run SPARQL query and collate results
errfound = False
err = NOERR
report = list()
results = g.query(query_term)

Analyse results of query.

In [3]:
# Error message building function
def report_append(err_new, msg):
    """
    Writes a report of an error and sets the error level appropriately.
    err: the error level to set
    msg: the message to write
    """
    global err
    global errfound
    global report
    if err_new > err:
        err = err_new
    errfound = True
    report.append(msg)

report.append('Properties omitted for these obsolete concepts:')

for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])
    obsolete_since  = str(r['obsolete_since'])
    deprecated = str(r['deprecated'])
    replacedBy = str(r['replacedBy'])
    consider = str(r['consider'])
    oldParent = str(r['oldParent'])
    deprecation_comment = str(r['deprecation_comment'])

    # Check for obsolete concepts
    if "obsolete" in subsetpath:
        if obsolete_since == "None":
            report_append(WARN, "WARN <obsolete_since> property missing ::: " + id +  ' (' + term + ')')
        if deprecated == "None":
            report_append(ERROR, "ERROR <owl:deprecated> property missing ::: " + id +  ' (' + term + ')')
        if replacedBy == "None" and consider == "None":
            report_append(ERROR, "ERROR Both <oboInOwl:replacedBy> and <oboInOwl:consider> missing ::: " + id +  ' (' + term + ')')
        if oldParent == "None":
            report_append(INFO, "INFO <oldParent> missing ::: " + id +  ' (' + term + ')')
        continue

Write report and return approriate value.

In [4]:
# Return exit code (raises exception)
if errfound:
    print('"Test name": ' + '"annotationDeprecationOmission", ' +\
          '"Status": ' + '"' + err + '", ' +\
          '"Reason": ' + '"' + '\n'.join(report) + '"')

    # print("\n".join(report))
    # sys.exit(err)
else:
    print('"Test name": ' + '"annotationDeprecationOmission", ' +\
          '"Status": ' + '"' + NOERR + '", ' +\
          '"Reason": ' + '"-"')

    # print("No issues found.")
    # sys.exit(NOERR)

"Test name": "annotationDeprecationOmission", "Status": "WARN", "Reason": "Properties omitted for these obsolete concepts:
INFO <oldParent> missing ::: http://edamontology.org/format_1620 (dbSNP polymorphism report format)
INFO <oldParent> missing ::: http://edamontology.org/operation_0332 (Immunogen design)
INFO <oldParent> missing ::: http://edamontology.org/format_1981 (GenBank feature)
INFO <oldParent> missing ::: http://edamontology.org/format_1698 (KEGG COMPOUND entry format)
INFO <oldParent> missing ::: http://edamontology.org/topic_0188 (Sequence profiles and HMMs)
INFO <oldParent> missing ::: http://edamontology.org/topic_0803 (Human disease)
INFO <oldParent> missing ::: http://edamontology.org/operation_2120 (Listfile processing)
INFO <oldParent> missing ::: http://edamontology.org/data_1325 (Protein features report (active sites))
INFO <oldParent> missing ::: http://edamontology.org/operation_1837 (Residue symmetry contact calculation)
INFO <oldParent> missing ::: http://eda